In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
#from catboost import CatBoostClassifier, CatBoostRegressor, metrics
from google.colab import drive
import os


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_dirty = pd.read_csv('/content/drive/MyDrive/creditcard.csv')

FileNotFoundError: ignored

In [ ]:
df_dirty.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


**Сделаем предобработку данных** заменим пропуски с помощью KNN

In [ ]:
imputer = KNNImputer()
df = pd.DataFrame(imputer.fit_transform(df_dirty), columns=df_dirty.columns)

In [ ]:
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0.0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0.0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0.0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0.0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0.0


**Решаем проблему балансировки классов** применим метод ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [ ]:
X, y = df.drop('Class', axis=1), df['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

In [ ]:
X_tr_tr, X_test_test, y_tr_tr, y_test_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

**Попробуем ансамблевые алгоритмы**

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score

In [ ]:
param_grid = {'n_estimators': [5000],
              'max_depth': [7],
              'learning_rate': [0.01],
            }

pipeline_catboost_class = make_pipeline(GridSearchCV(CatBoostClassifier(),
                                         param_grid=param_grid,
                                         cv=5,
                                         scoring='f1'
                                         ))

pipeline_catboost_class.fit(X_resampled, y_resampled)

In [ ]:
best_params = pipeline_catboost_class.named_steps['gridsearchcv'].best_params_
best_params

{'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 5000}

In [ ]:
print("Accuracy:", accuracy_score(pipeline_catboost_class.predict(X_test), y_test))
print("Presicion:", precision_score(pipeline_catboost_class.predict(X_test), y_test))
print("Recall:", recall_score(pipeline_catboost_class.predict(X_test), y_test))
print("ROC_AUC:", roc_auc_score(pipeline_catboost_class.predict(X_test), y_test))

Accuracy: 0.9992977774656788
Presicion: 0.8673469387755102
Recall: 0.7589285714285714
ROC_AUC: 0.8793499497424299


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt


# Иерархическая кластеризация с методом linkage ward
Z = linkage(X, method='ward')

# Построение дендрограммы
plt.figure(figsize=(15, 10))
dendrogram(Z, truncate_mode='level', p=5)
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

n_clusters = 2

# Выполнение кластеризации с количеством кластеров n_clusters
labels = fcluster(Z, n_clusters, criterion='maxclust')

silhouette_avg = silhouette_score(X, labels)

print("Silhouette score:", silhouette_avg)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

In [ ]:
params = {
    'learning_rate': [0.1],
    'max_depth': [5, 10],
    'subsample': [0.7],
}

skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc') # As the number of classes are 2

model_Xg = GridSearchCV(xgb_classifier,
                        param_grid=params,
                        scoring='roc_auc',
                        cv=skf,
                        n_jobs=-1,
                        verbose=1,
                        return_train_score=True)


model_Xg.fit(X_tr_tr, y_tr_tr)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='auc', feature_types=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [5, 10],
                         'subsample': [0.7]},
             return_train_score=True, scoring='roc_auc', verbose=1)

In [ ]:
print("Accuracy:", accuracy_score(model_Xg.predict(X_test_test), y_test_test))
print("Presicion:", precision_score(model_Xg.predict(X_test_test), y_test_test))
print("Recall:", recall_score(model_Xg.predict(X_test_test), y_test_test))
print("ROC_AUC:", roc_auc_score(model_Xg.predict(X_test_test), y_test_test))

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
params = {
    'learning_rate': [0.1],
    'max_depth': [5],
    'subsample': [0.5],
}

skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')

model_Xg = GridSearchCV(xgb_classifier,
                        param_grid=params,
                        scoring='roc_auc',
                        cv=skf,
                        n_jobs=-1,
                        verbose=1,
                        return_train_score=True)


model_Xg.fit(X_tr, y_tr)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='auc', feature_types=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1], 'max_depth': [5],
                         'subsample': [0.7]},
             return_train_score=True, scoring='roc_auc', verbose=1)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
from numpy import concatenate
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PowerTransformer

import xgboost as xgb

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/creditcard.csv')

In [ ]:
imputer = KNNImputer()
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

X, y = data.drop(['Class'], axis=1), data['Class']

X_s, X_t, y_s, y_t = train_test_split(X, y, test_size = 0.7, random_state = 42, stratify = y)

adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_s, y_s)
'''
df_s = pd.concat([X_resampled, y_resampled], axis=1)

X_s, y_s = df_s.drop(['Class'], axis=1), df_s['Class']
'''
pt= PowerTransformer(method='yeo-johnson', copy=True)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.7, random_state = 42, stratify = y_resampled)

pt.fit(X_train)

X_train_pt = pt.transform(X_train)
X_test_pt = pt.transform(X_test)

y_train_pt = y_train
y_test_pt = y_test

'''
X_train_lab, X_test_unlab, y_train_lab, y_test_unlab = train_test_split(X_train_pt, y_train_pt, test_size = 0.7, random_state = 42, stratify = y_train_pt)

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
cluster_labels = kmeans.labels_
'''

'\nX_train_lab, X_test_unlab, y_train_lab, y_test_unlab = train_test_split(X_train_pt, y_train_pt, test_size = 0.7, random_state = 42, stratify = y_train_pt)\n\nkmeans = KMeans(n_clusters=2, random_state=42)\nkmeans.fit(X)\ncluster_labels = kmeans.labels_\n'

In [ ]:
X_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
50265,44414.000000,1.328244,-0.510560,0.053151,-0.787074,-0.570897,-0.554529,-0.302116,-0.120469,-0.959277,...,0.110316,-0.260854,-0.977358,0.165800,-0.329269,0.150747,-0.594631,-0.000816,0.018762,49.900000
392461,57092.877126,-6.276232,3.608252,-6.674305,4.076992,-3.940252,-0.499706,-7.832027,-1.236131,-3.644496,...,0.760813,-1.059144,0.965654,0.438434,-0.534475,-0.058370,0.280751,0.466464,0.202509,0.554989
213325,139206.000000,-2.935527,3.186006,-0.950710,-2.840934,2.693587,-0.764270,4.083500,-2.975619,4.827349,...,3.271018,-1.030309,0.861569,-0.716117,-1.046866,0.361635,-0.213197,-0.324693,-2.008942,3.850000
247356,153550.000000,-0.592223,0.236797,0.959800,-0.706403,1.065852,2.191341,-0.122620,0.777309,0.766049,...,-0.008681,0.294890,1.237400,-0.183466,-2.340754,-0.392230,0.038908,0.502492,0.211944,25.000000
551151,155594.550530,-1.373688,3.667953,-6.052964,4.088442,-0.168661,-2.757177,-2.144142,0.844553,-3.549728,...,0.781953,0.709504,-0.069044,-0.248215,-0.440812,0.296556,0.470155,0.636892,0.321160,59.507247


In [ ]:
params = {
    'xgbclassifier__learning_rate': [0.8],
    'xgbclassifier__max_depth': [5],
    'xgbclassifier__subsample': [0.9]
}

skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')

model_Xg = make_pipeline(PowerTransformer(method='yeo-johnson', copy=True),
                         xgb_classifier)

model_Xg = GridSearchCV(model_Xg,
                        param_grid=params,
                        scoring='roc_auc',
                        cv=skf,
                        n_jobs=-1,
                        verbose=1,
                        return_train_score=True)


model_Xg.fit(X_train, y_train)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('powertransformer', PowerTransformer()),
                                       ('xgbclassifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='a...
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'xgbclassifier__learning_rate': [0.8],
                         'xgbclassifier__max_depth': [5],
                         'xgbclassifier__subsample': [0.9]},
             return_train_score=True, scoring='roc_auc', verbose=1)

In [ ]:
print("Accuracy:", accuracy_score(model_Xg.predict(X_test_pt), y_test))
print("Presicion:", precision_score(model_Xg.predict(X_test_pt), y_test))
print("Recall:", recall_score(model_Xg.predict(X_test_pt), y_test))
print("ROC_AUC:", roc_auc_score(model_Xg.predict(X_test_pt), y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


Accuracy: 0.5745356179756461


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


Presicion: 0.1490284757118928


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


Recall: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


ROC_AUC: 0.7701408935018867


In [ ]:
y_test.value_counts()

0.0    59706
1.0    59700
Name: Class, dtype: int64

In [ ]:
pt= PowerTransformer(method='yeo-johnson', copy=True)
pt.fit(X_t)
X_pt = pt.transform(X_t)

y_pt = y_t

In [ ]:
y_t.value_counts()

0.0    199021
1.0       344
Name: Class, dtype: int64

In [ ]:
print("Accuracy:", accuracy_score(model_Xg.predict(X_pt), y_t))
print("Presicion:", precision_score(model_Xg.predict(X_pt), y_t))
print("Recall:", recall_score(model_Xg.predict(X_pt), y_t))
print("ROC_AUC:", roc_auc_score(model_Xg.predict(X_pt), y_t))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


Accuracy: 0.999282722644396


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


Presicion: 0.7296511627906976


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


Recall: 0.8338870431893688


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


ROC_AUC: 0.9167099283784323


In [ ]:
pd.DataFrame(model_Xg.predict(X_pt)).value_counts()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


0    199064
1       301
dtype: int64

In [ ]:
pred = pd.DataFrame(model_Xg.predict(X_pt))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


In [ ]:
ans = pd.concat([X_t, pd.DataFrame({'pred': model_Xg.predict(X_pt)})], axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


In [ ]:
len(ans['pred'])

259175

In [ ]:
new_df = ans[0.0 != ans['pred']]

In [ ]:
new_df.to_csv('new_table.csv', index=False)

In [ ]:
new_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,pred
262307,160408.0,-0.472218,0.747697,2.244439,-0.195821,-0.354560,-0.818687,0.720575,-0.312265,0.306500,...,-0.084521,-0.004725,-0.010604,0.850714,-0.479498,-0.625194,-0.129662,-0.065652,12.49,NaN
227796,145226.0,2.083617,0.565970,-3.353498,0.514738,1.438395,-1.048235,0.638852,-0.237919,-0.073617,...,-0.047674,-0.002137,-0.084217,0.096758,0.376348,0.693083,-0.088667,-0.029355,0.76,NaN
5683,5946.0,1.016900,0.526658,-0.292481,1.380862,0.530523,0.084217,0.015508,0.094233,0.939839,...,-0.082616,-0.000116,-0.105041,-0.482348,0.468313,-0.328866,0.027916,0.042265,49.35,1.0
279393,168837.0,-0.914698,1.600272,-0.683617,-1.746796,1.750169,-0.946636,2.397652,-1.197813,1.393646,...,-0.149316,0.904673,-0.331897,0.698517,-0.208183,-0.072984,0.192037,-0.487096,0.77,NaN
215252,139991.0,2.017144,-0.324693,-0.489137,0.331919,-0.308376,0.001216,-0.590803,-0.012667,1.305594,...,0.178032,0.801200,0.107587,0.528348,0.003405,-0.233572,0.044782,-0.030741,4.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
193618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
193714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
196433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
len(new_df['pred'])

85743

In [ ]:
imputer = KNNImputer()
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

X, y = data.drop(['Class'], axis=1), data['Class']


X_s, y_s = train_test_split(X, y, test_size = 0.50, random_state = 42, stratify = y)


smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

df_s = pd.concat([X_resampled, y_resampled], axis=1)

df_sample = df_s.sample(n=14000, random_state=42)
X, y = df_sample.drop(['Class'], axis=1), df_sample['Class']

pt= PowerTransformer(method='yeo-johnson', copy=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50, random_state = 42, stratify = y)

pt.fit(X_train)

X_train_pt = pt.transform(X_train)
X_test_pt = pt.transform(X_test)

y_train_pt = y_train
y_test_pt = y_test


X_train_lab, X_test_unlab, y_train_lab, y_test_unlab = train_test_split(X_train_pt, y_train_pt, test_size = 0.50, random_state = 42, stratify = y_train_pt)

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
cluster_labels = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
X_train_mixed = concatenate((X_train_lab, X_test_unlab))
# Присвоение меток кластеров объектам, для которых изначально нет меток
nolabel = [-1 for _ in range(len(y_test_unlab))]

y_train_mixed = concatenate((y_train_lab, nolabel))

model = LabelPropagation(n_jobs=-1)

model.fit(X_train_mixed, y_train_mixed)

tran_labels = model.transduction_

params = {
    'learning_rate': [0.1],
    'max_depth': [5, 7],
    'subsample': [0.5]
}

skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')

model_Xg = GridSearchCV(xgb_classifier,
                        param_grid=params,
                        scoring='roc_auc',
                        cv=skf,
                        n_jobs=-1,
                        verbose=1,
                        return_train_score=True)


model_Xg.fit(X_train_mixed, tran_labels)

y_test_pred = model_Xg.predict(X_test)

label_prop = LabelPropagation(kernel='knn', n_neighbors=5, n_jobs=-1)

label_prop.fit(X_train_mixed, y_test_pred)
y_test_prop = label_prop.transduction_

# Оценка качества
print('Accuracy:', accuracy_score(y_test, y_test_pred))
print('Precision:', precision_score(y_test, y_test_pred))
print('Recall:', recall_score(y_test, y_test_pred))
print('ROC AUC:', roc_auc_score(y_test, y_test_pred))

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Accuracy: 0.5977142857142858
Precision: 0.5556256903897744
Recall: 1.0
ROC AUC: 0.5952860017246335
